# Base Code

In [ ]:
#Path for the lusa news dataset
path_lusa=""

In [ ]:
test_set_ids = ['lusa_97',
 'lusa_4',
 'lusa_67',
 'lusa_20',
 'lusa_83',
 'lusa_104',
 'lusa_80',
 'lusa_79',
 'lusa_34',
 'lusa_47',
 'lusa_30',
 'lusa_96',
 'lusa_11',
 'lusa_112',
 'lusa_100',
 'lusa_77',
 'lusa_38',
 'lusa_86',
 'lusa_60']

In [ ]:
import os

In [ ]:
files=list()
for i in os.listdir(path_lusa):
  if(i.startswith("lusa")):
    files.append(i.split(".")[0])


In [ ]:
files=set(files)

In [ ]:
train_set_ids=[f for f in files if f not in test_set_ids]

In [ ]:
train_set_ids

In [ ]:
#The annotation types used in the paper
ann_types=["Event",
           "Participant",
           "Time",
           "Spatial_Relation",
           "TLINK",
           "SRLINK",
           "QSLINK",
           "OLINK"
           ]

##See the documents with Counts for the Different Annotations

Used to determine what document would be used for the one shot generative approach

In [ ]:

#Function to compile the number of annotation of a given file
from collections import Counter
def getAnnCount(file="lusa_10.ann"):
  anns=list()
  with open(os.path.join(path_lusa,file),"r") as f:
     for line in f:
        a=line.split("\t")[1].split(" ")[0]
        for an_t in ann_types:
          if a.startswith(an_t):
            anns.append(a)
  count=dict(Counter(anns))
  count["id"]=file
  return count

In [ ]:
#iterate through all the files and generate a dataset

result=list()
for doc in train_set_ids:
  stat=getAnnCount(doc+".ann")
  #print(stat["id"])
  result.append(stat)



In [ ]:
import pandas as pd

df=pd.DataFrame.from_dict(result)

In [ ]:
df

# Entity extraction prompt using lang extract

In [ ]:
!pip install langextract

In [ ]:
import langextract as lx

In [ ]:
prompt_description = f'''Tarefa

Analisa o texto abaixo (em Português Europeu) e extrai eventos, participantes, expressões temporais (TIMEX) e relações espaciais, respeitando rigorosamente as instruções seguintes.
Não omitas informação relevante nem introduzas elementos que não estejam explicitamente presentes no texto.

1. Eventos

Identifica todas as situações temporalmente relevantes, incluindo:

verbos principais (exclui verbos auxiliares),

nominalizações com interpretação de acontecimento ou de estado,

adjetivos predicativos que exprimem estados relevantes,

expressões (semi)lexicalizadas que funcionem como predicador.

Regras de decisão:

Verbos aspetuais (ex. começar, continuar, terminar) devem ser identificados como eventos autónomos e associados ao evento principal.

Em construções copulativas, marca o adjetivo ou o nome (o predicativo do sujeito) que estiver a seguir ao verbo copulativo.

Em construções com ir + infinitivo:

se os verbos forem adjacentes, identifica apenas o verbo principal;

se não forem adjacentes, identifica ambos como eventos distintos.

Em sintagmas preposicionais, identifica apenas o núcleo nominal.

Exclui nominalizações que não tenham leitura eventiva.

2. Expressões temporais (TIMEX)

Identifica todas as expressões que denotem:

tempo de calendário,

hora do dia,

duração,

frequência temporal.

Inclui apenas:

sintagmas nominais temporais (ex. o verão passado, três dias),

advérbios temporais (ex. ontem, recentemente).

Exclui:

preposições e conetores temporais (durante, quando, enquanto),

expressões vagas não quantificáveis (frequentemente, raramente),

nomes próprios usados como datas (25 de Abril).

3. Participantes

Identifica todas as entidades relevantes envolvidas nos eventos, incluindo:

sintagmas nominais completos (com determinantes, modificadores e complementos),

pronomes com referência a entidades do discurso.

Cada participante deve corresponder a uma entidade concreta ou abstrata que intervenha, seja afetada ou seja localizada por um evento.

4. Relações espaciais

Identifica todas as expressões que codificam relações espaciais, quer de localização estática quer de movimento, incluindo:

sintagmas preposicionais ou adverbiais (ex. em, para, desde, perto de, ao longo de).

Sempre que possível, distingue:

relações de localização,

relações de direção,

relações de origem, percurso ou destino.

Cada item deve ter um identificador único.

Mantém consistência entre elementos referenciados.

Não inventes informação nem faças inferências não suportadas pelo texto.

'''

In [ ]:
#text from the few shot document chosen (lusa_106)
text_few_shot=f'''
Porto, 16 dez 2020
VSYM (PM) // JAP
Grua de camião derrubou poste de iluminação no Porto e matou homem de 52 anos
Um homem de 52 anos morreu hoje, e um outro sofreu ferimentos ligeiros, na sequência de um acidente com um camião que derrubou um poste de eletricidade na Rua da Constituição, no Porto, disse à Lusa fonte da PSP.
De acordo com a fonte das Relações Públicas do Comando Metropolitano da PSP do Porto, o poste de iluminação foi derrubado pela grua do camião atingindo o homem que circulava na rua.
A vítima ainda foi transportada com vida para o Hospital de Santo António, mas acabou por morrer.
A estrutura atingiu ainda o funcionário de umas bombas de gasolina, que sofreu ferimentos ligeiros.
O acidente ocorreu cerca das 07:50.
Em comunicado, a Câmara do Porto, esclareceu, entretanto, que o acidente que provocou a queda de um poste de eletricidade envolveu um veículo da Empresa Municipal de Ambiente do Porto e já decidiu averiguar o que terá acontecido.
"A empresa ordenou já a abertura de um processo de averiguação interna às causas deste acidente estando também, naturalmente, disponível para colaborar com as autoridades", assinala o município lamentando a morte e endereçando as condolências à sua família.
'''

In [ ]:
#Function to get the annotations for the few shot example and convert them in the lang extract Extraction object. Returns a list of all annotations
def getFewShotExamples(file="lusa_106"):
  fs=list()
  with open(os.path.join(path_lusa,file+".ann"),"r") as f:
     for line in f.read().splitlines():
        atrs=line.split("\t")
        cls=atrs[1].split(" ")[0]

        if cls in ann_types[0:4]:# apenas entidades
             #print(atrs)
             text=atrs[2]
             fs.append(lx.data.Extraction(extraction_class=cls, extraction_text=text),)
  return fs

In [ ]:
d=getFewShotExamples()
print(d)

[Extraction(extraction_class='Time', extraction_text='16 dez 2020', char_interval=None, alignment_status=None, extraction_index=None, group_index=None, description=None, attributes=None), Extraction(extraction_class='Event', extraction_text='derrubou', char_interval=None, alignment_status=None, extraction_index=None, group_index=None, description=None, attributes=None), Extraction(extraction_class='Event', extraction_text='matou', char_interval=None, alignment_status=None, extraction_index=None, group_index=None, description=None, attributes=None), Extraction(extraction_class='Participant', extraction_text='Um homem de 52 anos', char_interval=None, alignment_status=None, extraction_index=None, group_index=None, description=None, attributes=None), Extraction(extraction_class='Event', extraction_text='morreu', char_interval=None, alignment_status=None, extraction_index=None, group_index=None, description=None, attributes=None), Extraction(extraction_class='Time', extraction_text='hoje', 

In [ ]:
import langextract as lx
import json
"""
HERE SHOULD BE THE KEY
"""
LANGEXTRACT_API_KEY="" #place Google AI Studio Gemini Key here

"""
HERE SHOULD BE THE KEY
"""
#lusa_106 -> used as fewshot

def extract_info_gemini(input_text,prompt_description):
    example_text=text_few_shot
  # Define extraction prompt
    prompt_description = prompt_description

      # Define example data with entities in order of appearance
    examples = [
          lx.data.ExampleData(
              text=example_text,
              extractions=getFewShotExamples()
          )
    ]
    result = lx.extract(
        text_or_documents=input_text,
        prompt_description=prompt_description,
        examples=examples,
        api_key=LANGEXTRACT_API_KEY,
        model_id="gemini-2.5-flash-lite",
    )
    return result


### Testing

In [ ]:
# test example
test_example=train_set_ids[10]
print(test_example)
with open(os.path.join(path_lusa,test_example+".txt"),"r") as f:
  test_text=f.read()

result=extract_info_gemini(test_text,prompt_description)

lusa_22


In [ ]:
# Save the results to a JSONL file
lx.io.save_annotated_documents([result], output_name="extraction_results.jsonl", output_dir=".")

# Generate the visualization from the file
html_content = lx.visualize("extraction_results.jsonl")
with open("visualization.html", "w", encoding="utf-8") as f:
    if hasattr(html_content, 'data'):
        f.write(html_content.data)  # For Jupyter/Colab
    else:
        f.write(html_content)

LangExtract: Saving to extraction_results.jsonl: 1 docs [00:00, 220.43 docs/s]

✓ Saved 1 documents to extraction_results.jsonl



LangExtract: Loading extraction_results.jsonl: 100%|██████████| 17.4k/17.4k [00:00<00:00, 9.42MB/s]

✓ Loaded 1 documents from extraction_results.jsonl


##FULL TEST DATA

In [ ]:
test_set_ids

['lusa_97',
 'lusa_4',
 'lusa_67',
 'lusa_20',
 'lusa_83',
 'lusa_104',
 'lusa_80',
 'lusa_79',
 'lusa_34',
 'lusa_47',
 'lusa_30',
 'lusa_96',
 'lusa_11',
 'lusa_112',
 'lusa_100',
 'lusa_77',
 'lusa_38',
 'lusa_86',
 'lusa_60']

In [ ]:
test_set_ids[13]


'lusa_112'

In [ ]:
output_folder="results/entities_outputs" # the output folder where to place the result files
import time
#time.sleep(60*60)
for test_doc in test_set_ids:
  #time.sleep(60*30)
  print(test_doc)
  with open(os.path.join(path_lusa,test_doc+".txt"),"r") as f:
    test_text=f.read()
  result=extract_info_gemini(test_text,prompt_description)
  lx.io.save_annotated_documents([result], output_name=os.path.join(output_folder,"test_results",test_doc+"_results.jsonl"), output_dir=".")

  # Generate the visualization from the file
  html_content = lx.visualize(os.path.join(output_folder,"test_results",test_doc+"_results.jsonl"))
  with open(os.path.join(output_folder,"test_results",test_doc+"_visualization.html"), "w", encoding="utf-8") as f:
      if hasattr(html_content, 'data'):
          f.write(html_content.data)  # For Jupyter/Colab
      else:
          f.write(html_content)

#Relation Extraction prompt usinge Google GenerativeAI library

In [ ]:
#Function to get the  Relations from an annotation file to used as few-shot
def getFewShotRelationExamples(file="lusa_106",json=False):
  fs=list()
  with open(os.path.join(path_lusa,file+".ann"),"r") as f:
     for line in f.read().splitlines():
        atrs=line.split("\t")
        cls=atrs[1].split(" ")[0]
        cls=cls.split("_")[0] #separate TLINK de TLINK_identity
        if cls in ann_types[4:]:# apenas entidades
            args=atrs[1].split(" ")[1:]
            fs.append((cls,args[0].split(":")[1],args[1].split(":")[1]))

  #map arguments
  map_t=dict()
  with open(os.path.join(path_lusa,file+".ann"),"r") as f:
     for line in f.read().splitlines():
        entry=line.split("\t")
        id=entry[0]
        if id.startswith("T"):
          inout=entry[1].split(" ")
          begin=inout[1]
          end=inout[2]
          #print(entry)
          if json:
            map_t[id]={'begin': begin, 'end': end, 'span':entry[2]}
          else:
            map_t[id]=f'begin: {begin}, end: {end}, span:{entry[2]}'


  fs_mapped=list()
  for cl,arg1,arg2 in fs:
    arg1_m=map_t[arg1]
    arg2_m=map_t[arg2]
    if(json):
      fs_mapped.append({"class":cl,"arg1":arg1_m,"arg2":arg2_m})
    else:

      fs_mapped.append(f'{{class: {cl}, arg1:{{{arg1_m}}}, arg2:{{{arg2_m}}}}}')




  if json:
    return(fs_mapped)

  return "\n".join(fs_mapped)


In [ ]:
#Test it
s=getFewShotRelationExamples("lusa_83",json=True)
s

In [ ]:

path_results_entities="" #folder where the results for the entities were saved (same as output folder above)

#function to get the Entities from the previous extraction step for each document
def getEntities2Consider(file="lusa_100",d=False):
  file_ext=file+"_results.jsonl"


  with open(os.path.join(path_results_entities,file_ext)) as f:
      first_line = f.readline()
      result_entity=json.loads(first_line)
  entities_2_consider=list()
  for entry in result_entity["extractions"]:
    temp=dict()
    #print(entry)
    try:
      temp["class"]=entry["extraction_class"]
      temp["text"]=entry["extraction_text"]
      temp["begin"]=entry["char_interval"]["start_pos"]
      temp["end"]=entry["char_interval"]["end_pos"]
      if not d:

        str_out=f'classe: {temp["class"]}, texto: {temp["text"]}, begin: {temp["begin"]}, end: {temp["end"]}'
        entities_2_consider.append(str_out)
      else:
        entities_2_consider.append(temp)
    except Exception as e:
      print("Failed to extract entity to consider ", str(e))
  return entities_2_consider


In [ ]:
#testing
import json
getEntities2Consider("lusa_4",True)

In [ ]:
#Function to build the prompt. It receives the few shot text document, the relation annotations from that document and the test document.
#The entities for the test document are integrated via the test_doc id passed as an argument

def getRelationPrompt(text_few_shot,fs,test_doc):

  with open(os.path.join(path_lusa,test_doc+".txt"),"r") as f:
    test_text=f.read()


  prompt_relations=f'''
 Tarefa
Recebes:

um texto em Português Europeu;
as anotações já fornecidas de eventos, participantes, expressões temporais (TIMEX) e relações espaciais, cada uma identificada pelo respetivo span textual.

O teu objetivo é extrair ligações explícitas entre essas entidades.

Extrai apenas as seguintes classes de links:

TLINK — ligação temporal

OLINK — ligação referencial/objectal

QSLINK — ligação espacial qualitativa

SRLINK — ligação semântica entre evento e participante ou entre participante e participante

Restrições

Usa exclusivamente o texto literal das entidades anotadas como argumentos.

Não inferir relações: cria links apenas quando a ligação é claramente expressa no texto.

Produz apenas triplos, sem qualquer explicação adicional.

1) TLINK (Temporal Link)

Cria um TLINK quando o texto estabelece explicitamente uma relação temporal entre:

evento <-> evento

evento <-> expressão temporal

expressão temporal<-> expressão temporal

Direção do link

evento <-> tempo -> arg1 = evento, arg2 = expressão temporal

evento <-> evento ou tempo <-> tempo ->
arg1 = elemento mais recente no texto,
arg2 = elemento anterior


2) OLINK (Objectal Link)

Cria um OLINK quando duas menções referem explicitamente:

a mesma entidade (correferência),

uma relação parte–todo,

uma relação membro–conjunto ou subconjunto–conjunto,

uma disjunção referencial explícita.

Direção do link

arg1 = menção dependente (ex. pronome, descrição anafórica)

arg2 = menção antecedente



3) QSLINK (Qualitative Spatial Link)

Cria um QSLINK quando uma relação espacial estática liga explicitamente:

uma figura (evento ou participante localizado),

a um ground (local).

Direção do link

arg1 = figura

arg2 = ground


4) SRLINK (Semantic Role Link)

Cria um SRLINK quando um evento tem um participante (ou outro evento) como argumento explícito no texto.

Direção do link

arg1 = evento

arg2 = participante (ou evento subordinado)


Formato final de saída


Apenas relações no formato (classe: tipo da relação, arg1: {{begin, end, span}}, arg2: {{begin, end, span}}

Uma relação por linha

Nenhum texto adicional antes ou depois

Texto Exemplo:
{text_few_shot}

Relações de Exemplo: {fs}

Texto a considerar: {test_text}

Entidades anotadas: {getEntities2Consider(test_doc)}

Relações:

  '''
  return prompt_relations

In [ ]:
#insatll dependencies
!pip install -U google-generativeai
import google.generativeai as genai
import os

In [ ]:
#if you want to run jus the relations load the API Key here and uncomment the variable
#LANGEXTRACT_API_KEY=""

def promptGEMINI(prompt):
  genai.configure(api_key=LANGEXTRACT_API_KEY)

  model = genai.GenerativeModel(
      model_name="gemini-2.5-flash-lite",
      generation_config={
          "temperature": 0.7,
          "top_p": 0.95
      }
  )


  response = model.generate_content(prompt)

  # 4. Print the results
  print("-" * 30)
  print(f"Response:\n{response.text}")
  print("-" * 30)
  return response

In [ ]:
relations_results_path="" #path to the result folder (where to put the generated results)


#function to get the relations results from gemini given a test document and a few shot document

def getRelationsFromGemini(test_doc,fs_doc="lusa_106"):
  with open(os.path.join(path_lusa,fs_doc+".txt"),"r") as f:
    text_few_shot=f.read()
  fs=getFewShotRelationExamples(fs_doc)

  prompt=getRelationPrompt(text_few_shot=text_few_shot,fs=fs,test_doc=test_doc)
  response=promptGEMINI(prompt)

  filename=test_doc+"rel.txt"
  with open(os.path.join(relations_results_path,filename), "w", encoding="utf-8") as f:
    f.write(response.text)
  return response



##RELATIONS FULL DATA

In [ ]:
import time
#time.sleep(60*60)
for test_doc in test_set_ids:
  print(test_doc)
  getRelationsFromGemini(test_doc)

#Consistency Analysis

Not included in the paper but it allows an analysis on the coverage of the relation extraction process on the usage of the ner extracted. Simply put it, it allows to see how many entities were correctly mapped from the ner extraction to the relations based on exact span matching.

In [ ]:
relation_folder="" # the relation results folder
entities_folder="" # the entities results folder

In [ ]:
def fix_to_json_line(line: str) -> str:
    line = line.strip()
    if not line:
        return line

    line = re.sub(r'([{\s,])([A-Za-z_]\w*)\s*:', r'\1"\2":', line)


    line = re.sub(
        r'("span"\s*:\s*)([^,}]+)',
        lambda m: m.group(1) + json.dumps(m.group(2).strip(),ensure_ascii=False),
        line
    )

    line = re.sub(r'("class"\s*:\s*)([A-Za-z_]\w*)', r'\1"\2"', line)

    return line

In [ ]:
import re
import ast
def getRelationResult(doc="lusa_100",complete=False):
  res=list()
  with open(os.path.join(relation_folder,doc+"rel.txt")) as f:
    lines = [line.rstrip() for line in f]
  res=list()
  for l in lines:
    try:
      d=fix_to_json_line(l)
      d=json.loads(d)
      d=dict(d)

      if complete:
        res.append(d)
      else:
        b=d["arg1"]["begin"]

        e=d["arg1"]["end"]

        span=d["arg1"]["span"]
        res.append((b,e,span))
        b=d["arg2"]["begin"]
        e=d["arg2"]["end"]
        span=d["arg2"]["span"]

        res.append((b,e,span))
    except Exception as e:
      print("error " + str(e))
      print(l)

  return res


In [ ]:
count=0
size=0
for test_doc in test_set_ids:
  ent=getEntities2Consider(test_doc,d=True)
  rel=getRelationResult(test_doc)
  size=size+len(rel)
  for (b,e,span) in rel:
    for entry in ent:
      #print(entry)
      if b==entry["begin"] and e==entry["end"]:
        count=count+1
        break

In [ ]:
print("Relations linked with the extracted entities:", count)
print("Total Relations Extracted:",size)
print("Accuracy on correct Relation-Entity Link:", count/size)



#Evaluation

Evaluation of the results

In [ ]:
import os
import json

In [ ]:
#Testing functions
test_doc="lusa_100"
predictions=getRelationResult(test_doc,complete=True)
ground_truth=getFewShotRelationExamples(test_doc,json=True)


In [ ]:
#util function for format conversion
def convertJson2Tuple(list_gt):
  res=list()
  for ind in list_gt:
    entry=(ind["class"],str(ind["arg1"]["begin"]),str(ind["arg1"]["end"]),str(ind["arg2"]["begin"]),str(ind["arg2"]["end"]))
    res.append(entry)
  return res


In [ ]:
#compute the metrics
def computeGTandPredBetter(ground_truth,predictions):

  list_gt=list()
  list_pred=list()
  total_tp = total_fp = total_fn = 0
  results=dict()


  labels=["TLINK","OLINK","SRLINK","QSLINK"]
  counts = {label: {"tp": 0, "fp": 0, "fn": 0,"sup":0} for label in labels}

  for test_doc in test_set_ids:
    predictions=getRelationResult(test_doc,complete=True)
    ground_truth=getFewShotRelationExamples(test_doc,json=True)
    predictions=set(convertJson2Tuple(predictions))
    ground_truth=set(convertJson2Tuple(ground_truth))



    for label in labels:
        #filter by label
        pred_label = {ann for ann in predictions if ann[0] == label}
        gold_label = {ann for ann in ground_truth if ann[0] == label}

        counts[label]["tp"] += len(pred_label & gold_label)
        counts[label]["fp"] += len(pred_label - gold_label)
        counts[label]["fn"] += len(gold_label - pred_label)
        counts[label]["sup"] += len(gold_label)



  for label in labels:
    tp, fp, fn = counts[label]["tp"], counts[label]["fp"], counts[label]["fn"]
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    results[label] = {"precision": precision, "recall": recall, "f1": f1}
    total_tp += tp
    total_fp += fp
    total_fn += fn

       # Micro-average across both labels
  micro_p = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
  micro_r = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
  micro_f1 = 2 * micro_p * micro_r / (micro_p + micro_r) if (micro_p + micro_r) > 0 else 0.0

  results["micro_avg"] = {"precision": micro_p, "recall": micro_r, "f1": micro_f1}
  print(list(predictions)[0],list(ground_truth)[0])
  return results

In [ ]:
results=computeGTandPredBetter(ground_truth, predictions)
